In [1]:
import flexpolyline as fp
import geopandas as gpd
import logging
import pandas as pd
import psycopg2
import requests
import shapely

In [2]:
def open_connection():
    conn = \
        psycopg2.connect( \
            host = 'localhost',
            database = 'queroponto',
            user = 'postgres',
            password = 'postgres')

    cur = conn.cursor()
    
    return conn, cur

In [3]:
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [4]:
def get_ia(origin):
    payload = {
        'apiKey': 'KDUbLze9c55M07_jhPSR750KYGk9UxfX42DItWJWV8Y',
        'origin': origin,
        'range[type]': 'time',
        'range[values]': '300',
        'transportMode': 'car'
    }

    req = requests.get('https://isoline.router.hereapi.com/v8/isolines', params = payload)
    response = req.json()

    for item in response['isolines']:
        polygon = fp.decode(item['polygons'][0]['outer'])
        coords = [[coord[1], coord[0]] for coord in polygon]

    gdf = gpd.GeoDataFrame(geometry = [shapely.Polygon(coords)], crs = 'EPSG:4326')

    return gdf

In [5]:
def get_sectors(conn, cur, polygon):
    query = f" \
        SELECT SC.cod_ibge \
        FROM PUBLIC.setores_censitarios AS SC \
        WHERE ST_Intersects(SC.geom, \'SRID=4326;{polygon}\');"
    
    try:
        cur.execute(query)
        result = [row[0] for row in cur.fetchall()]
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

In [6]:
def get_data(conn, cur, sectors):
    sectors = ', '.join(map(str, [sector for sector in sectors]))
    
    query = f" \
        SELECT \
            SC.id_municipio, \
            SC.cod_ibge, \
            PRM.\"00_04\", PRM.\"05_09\", PRM.\"10_14\", PRM.\"15_19\", PRM.\"20_24\", PRM.\"25_29\", PRM.\"30_34\", \
            PRM.\"35_39\", PRM.\"40_44\", PRM.\"45_49\", PRM.\"50_54\", PRM.\"55_59\", PRM.\"60_64\", PRM.\"65_69\", \
            PRM.\"70_74\", PRM.\"75_79\", PRM.\"80_84\", PRM.\"85_89\", PRM.\"90_94\", PRM.\"95_99\", PRM.\"100\",   \
            PRF.\"00_04\", PRF.\"05_09\", PRF.\"10_14\", PRF.\"15_19\", PRF.\"20_24\", PRF.\"25_29\", PRF.\"30_34\", \
            PRF.\"35_39\", PRF.\"40_44\", PRF.\"45_49\", PRF.\"50_54\", PRF.\"55_59\", PRF.\"60_64\", PRF.\"65_69\", \
            PRF.\"70_74\", PRF.\"75_79\", PRF.\"80_84\", PRF.\"85_89\", PRF.\"90_94\", PRF.\"95_99\", PRF.\"100\",   \
            RS.\"00_04\", RS.\"05_09\", RS.\"10_14\", RS.\"15_19\", RS.\"20_24\", RS.\"25_29\", RS.\"30_34\", \
            RS.\"35_39\", RS.\"40_44\", RS.\"45_49\", RS.\"50_54\", RS.\"55_59\", RS.\"60_64\", RS.\"65_69\", \
            RS.\"70_74\", RS.\"75_79\", RS.\"80_84\", RS.\"85_89\", RS.\"90_94\", RS.\"95_99\", RS.\"100\",   \
            PIA.\"masculina\", PIA.\"feminina\", PIA.\"total\", \
            IE.\"masculino\", IE.\"feminino\", \
            DR.\"classe_ab\", DR.\"classe_c\", DR.\"classe_d\", DR.\"classe_e\", \
            RR.\"classe_a\", RR.\"classe_b\", RR.\"classe_c\", RR.\"classe_d\", RR.\"classe_e\" \
        FROM PUBLIC.setores_censitarios AS SC \
        JOIN PUBLIC.populacao_residente_mas AS PRM ON SC.cod_ibge = PRM.id_setor_censitario \
        JOIN PUBLIC.populacao_residente_fem AS PRF ON SC.cod_ibge = PRF.id_setor_censitario \
        JOIN PUBLIC.populacao_idade_ativa AS PIA ON SC.cod_ibge = PIA.id_setor_censitario \
        JOIN PUBLIC.razao_sexo AS RS ON SC.cod_ibge = RS.id_setor_censitario \
        JOIN PUBLIC.indice_envelhecimento AS IE ON SC.cod_ibge = IE.id_setor_censitario \
        JOIN PUBLIC.domicilio_renda AS DR ON SC.cod_ibge = DR.id_setor_censitario \
        JOIN PUBLIC.responsavel_renda AS RR ON SC.cod_ibge = RR.id_setor_censitario \
        WHERE SC.cod_ibge IN({sectors});"
        
    try:
        cur.execute(query)
        result = cur.fetchall()
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

---

In [7]:
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)s %(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [8]:
filename = 'loja-01'

In [9]:
origin = '-9.756869999999969,-36.65935070000001'   # loja-01
# origin = '-9.75257,-36.659500400000006'            # loja-02
# origin = '-10.28169279999999,-36.56296889999999'   # loja-03
# origin = '-9.731959199999986,-36.67763079999998'   # loja-04
# origin = '-9.759829900000007,-36.6605428'          # loja-05
# origin = '-10.291947200000006,-36.584740900000014' # loja-07
# origin = '-9.654918600000004,-35.73275409999996'   # loja-08
# origin = '-9.744153799999998,-36.65529809999997'   # loja-09
# origin = '-9.407226299999992,-36.63035589999999'   # loja-10
# origin = '-9.408816599999975,-36.63168619999999'   # loja-11
# origin = '-9.66127510000001,-35.702272699999995'   # loja-12
# origin = '-9.757350399999988,-36.666036'           # loja-14
# origin = '-9.78115669999999,-36.09609069999999'    # loja-15
# origin = '-9.5332118,-37.29475300000001'           # loja-17
# origin = '-9.762726999999984,-36.661038999999995'  # loja-18
# origin = '-9.385276199999982,-37.99815710000002'   # loja-21
# origin = '-9.407306500000011,-36.6316134'          # loja-22

In [12]:
%%time
polygon = get_ia(origin)

AttributeError: module 'shapely' has no attribute 'Polygon'

In [11]:
polygon = polygon['geometry'].to_wkt()[0]

NameError: name 'polygon' is not defined

In [ ]:
sectors = get_sectors(conn, cur, polygon)

In [ ]:
len(sectors)

In [ ]:
%%time
ia_data = get_data(conn, cur, sectors)

In [ ]:
close_connection(conn, cur)

In [ ]:
cols = [
    'ID_MUNICIPIO', 'ID_SETOR_CENSITARIO',
    'PRM_00_04', 'PRM_05_09', 'PRM_10_14', 'PRM_15_19', 'PRM_20_24', 'PRM_25_29', 'PRM_30_34',
    'PRM_35_39', 'PRM_40_44', 'PRM_45_49', 'PRM_50_54', 'PRM_55_59', 'PRM_60_64', 'PRM_65_69',
    'PRM_70_74', 'PRM_75_79', 'PRM_80_84', 'PRM_85_89', 'PRM_90_94', 'PRM_95_99', 'PRM_100',
    'PRF_00_04', 'PRF_05_09', 'PRF_10_14', 'PRF_15_19', 'PRF_20_24', 'PRF_25_29', 'PRF_30_34', 
    'PRF_35_39', 'PRF_40_44', 'PRF_45_49', 'PRF_50_54', 'PRF_55_59', 'PRF_60_64', 'PRF_65_69', 
    'PRF_70_74', 'PRF_75_79', 'PRF_80_84', 'PRF_85_89', 'PRF_90_94', 'PRF_95_99', 'PRF_100',
    'RS_00_04', 'RS_05_09', 'RS_10_14', 'RS_15_19', 'RS_20_24', 'RS_25_29', 'RS_30_34', 
    'RS_35_39', 'RS_40_44', 'RS_45_49', 'RS_50_54', 'RS_55_59', 'RS_60_64', 'RS_65_69', 
    'RS_70_74', 'RS_75_79', 'RS_80_84', 'RS_85_89', 'RS_90_94', 'RS_95_99', 'RS_100',
    'PIA_M', 'PIA_F', 'PIA_T',
    'IE_M', 'IE_F',
    'DR_CLASSE_AB', 'DR_CLASSE_C', 'DR_CLASSE_D', 'DR_CLASSE_E',
    'RR_CLASSE_A', 'RR_CLASSE_B', 'RR_CLASSE_C', 'RR_CLASSE_D', 'RR_CLASSE_E'
]

In [ ]:
df = pd.DataFrame(data = ia_data, columns = cols)

In [ ]:
len(df)

In [ ]:
df.to_csv(f"{filename}.csv", index = False)